https://machinelearningmastery.com/imbalanced-classification-with-the-fraudulent-credit-card-transactions-dataset/

In [1]:
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = 'all'

In [2]:
import pandas as pd

pd.set_option( 'display.max_columns' , None ) 

In [3]:
import requests

import os

In [4]:
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/creditcard.csv.zip'

myfile = requests.get( url )

open( os.getcwd() + '\\' + 'creditcard.csv.zip' , 'wb' ).write( myfile.content )

68787176

In [5]:
import zipfile

In [6]:
with zipfile.ZipFile( os.getcwd() + '\\' + 'creditcard.csv.zip' , 'r' ) as zip_ref :
    
    zip_ref.extractall( os.getcwd() )

In [7]:
df = pd.read_csv(os.getcwd() + '\\' + 'creditcard.csv' , header = None )

In [8]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [9]:
print( df.shape )

(284807, 31)


In [10]:
target = df.values[ : , -1 ]

In [11]:
from collections import Counter

In [12]:
counter = Counter( target )

print( counter )

Counter({0.0: 284315, 1.0: 492})


In [13]:
for k , v in counter.items():

    p = v / len( target ) * 100

    print( f'class : {k} | count : {v} | % : {p:.2f} ' )

class : 0.0 | count : 284315 | % : 99.83 
class : 1.0 | count : 492 | % : 0.17 


In [14]:
data = df.values

X = data[ : , :-1 ]

y = data[ : , -1 ]

print( X.shape )

print( y.shape )

(284807, 30)
(284807,)


# Model Test and Baseline Result

In [15]:
from sklearn.metrics import precision_recall_curve

from sklearn.metrics import auc

In [16]:
def precision_recall_auc ( y_true , probas_pred ) :

    precision , recall , _ = precision_recall_curve( y_true , probas_pred ) 
    
    return auc( recall , precision )

In [17]:
from sklearn.model_selection import RepeatedStratifiedKFold

#https://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html?highlight=make_scorer
from sklearn.metrics import make_scorer

from sklearn.model_selection import cross_val_score

In [18]:
def evaluate_model( X , y , model ) :
    
    cv = RepeatedStratifiedKFold( n_splits = 10 , n_repeats = 3 , random_state = 1 )

    metric = make_scorer( precision_recall_auc , needs_proba = True )

    scores = cross_val_score( model , X , y , scoring = metric , cv = cv , n_jobs = -1 )

    return scores

In [19]:
from sklearn.dummy import DummyClassifier

from numpy import mean

from numpy import std

In [20]:
model = DummyClassifier( strategy = 'constant' , constant = 1 )

In [21]:
scores = evaluate_model( X , y , model )

print( f'precision recall auc : {mean(scores):.2f} ( {std(scores):.2f} )' )

precision recall auc : 0.50 ( 0.00 )


In [22]:
from sklearn.preprocessing import StandardScaler

from sklearn.pipeline import Pipeline

from sklearn.tree import DecisionTreeClassifier

from sklearn.neighbors import KNeighborsClassifier

from sklearn.ensemble import RandomForestClassifier

from sklearn.ensemble import ExtraTreesClassifier

from sklearn.ensemble import BaggingClassifier

In [23]:
def get_models():

    models = dict()

    models[ 'CART' ] = DecisionTreeClassifier()
    
    steps = [ ( 'step - standard scaler' , StandardScaler() ) , ( 'step - knn' , KNeighborsClassifier() ) ]
    models[ 'KNN' ] = Pipeline( steps = steps )
    
    models[ 'BAGGING' ] = BaggingClassifier( n_estimators = 100 )
    
    models[ 'RANDOM FOREST' ] = RandomForestClassifier( n_estimators = 100 )

    models[ 'EXTRA TREE'] = ExtraTreesClassifier( n_estimators = 100 ) 
    
    return models 

In [ ]:
models = get_models()

results = list()

names = list()


for name , model in models.items() :
    
    names.append( name )
    
    scores = evaluate_model( X , y , model ) 
    
    results.append( scores )
    
    print( f' model : {name} : {mean(scores):.3f} ( {std(scores):.3f} )' )
    
    

 model : CART : 0.760 ( 0.034 )
 model : KNN : 0.869 ( 0.042 )


In [ ]:
pyplot.boxplot( results , labels = names , showmeans = True )

pyplot.grid( axis = 'y' , color = 'black' , linestyle = '-.' , linewidth = 0.5 )

pyplot.xticks( rotation = 90 )

pyplot.show()